# 커스텀 데이터를 활용하여 YOLOv8로 클라이밍 이미지 Object Detection

Roboflow를 활용해 클라이밍 이미지에서 얼굴, 손, 발을 찾기 위한 데이터셋을 만들고,

YOLO V8을 사용하여 앞서 만든 데이터셋으로 모델 학습과 예측을 한다.

## 1.데이터셋

* 데이터셋 라이브러리 설치하기

In [1]:
!pip install roboflow

* 데이터셋 라이브러리 불러오기

In [2]:
from roboflow import Roboflow

* 데이터셋 설치하기

In [3]:
!pip install ultralytics==8.0.196

In [4]:
rf = Roboflow(api_key="endhVMMcQKgXD6F3PjMx")
project = rf.workspace("aivle5-fhpx3").project("climbing-bcony")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Climbing-1 in yolov8:: 100%|██████████| 508/508 [00:00<00:00, 3159.77it/s]


## 2.모델링

* 모델링 라이브러리 설치하기

In [5]:
!pip install ultralytics

* 모델링 라이브러리 불러오기

In [6]:
from ultralytics import YOLO, settings

* 데이터셋 경로 수정 & YAML 파일 경로 수정

In [7]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [8]:
import yaml

# yaml 파일 읽고 수정 필요

### 1) 모델 구조만 빌려와서 사용해보기

- 모델 구조 선택하기

In [9]:
model1 = YOLO(model='yolov8n.yaml', task='detect')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

- 모델 학습하기

In [10]:
model1.train(data='/content/Climbing-1/data.yaml',
            epochs=150,
            patience=7,
            pretrained=False,
            verbose=True,
            seed=1234,
            )

New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/Climbing-1/data.yaml, epochs=150, patience=7, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=1234, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=F

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7933f39313c0>
fitness: 0.41146580280822254
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.42399,     0.16871,     0.52679])
names: {0: 'foot', 1: 'hand', 2: 'head'}
plot: True
results_dict: {'metrics/precision(B)': 0.804447501342211, 'metrics/recall(B)': 0.6698325163398692, 'metrics/mAP50(B)': 0.7561812874128325, 'metrics/mAP50-95(B)': 0.3731640822965992, 'fitness': 0.41146580280822254}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.2079010009765625, 'inference': 2.096482685634068, 'loss': 0.0010850478191764988, 'postprocess': 1.6672367952307876}

- 예측해보기

In [11]:
results = model1.predict(source='/content/Climbing-1/test/images/*',
                        save=True,
                        line_width=2,
                        stream=True
                         )

In [12]:
for r in results :
    r_bbox = r.boxes


image 1/25 /content/Climbing-1/test/images/u106_jpg.rf.0e50dad5b2cdf3cb1c705c2a34c924e8.jpg: 640x640 2 foots, 2 heads, 8.6ms
image 2/25 /content/Climbing-1/test/images/u110_jpg.rf.78349a8d5d1d2f25d5fcd45eb87d0615.jpg: 640x640 1 foot, 1 hand, 2 heads, 8.4ms
image 3/25 /content/Climbing-1/test/images/u112_jpg.rf.a2a70d71b82ed44f4e33a1a0e53e52c5.jpg: 640x640 1 foot, 3 hands, 1 head, 8.4ms
image 4/25 /content/Climbing-1/test/images/u12_jpg.rf.3135fbab1b8ae760b418389e52d94449.jpg: 640x640 1 foot, 1 head, 9.9ms
image 5/25 /content/Climbing-1/test/images/u21_jpg.rf.4ae71f1edcdb66703710e1496a2e4e1d.jpg: 640x640 1 foot, 1 head, 8.2ms
image 6/25 /content/Climbing-1/test/images/u22_jpg.rf.4eb7505c344be9bca3ea3d56afb76a3d.jpg: 640x640 3 foots, 2 hands, 1 head, 8.0ms
image 7/25 /content/Climbing-1/test/images/u27_jpg.rf.3aa0e2a0c11944115ed8e212f9aefe73.jpg: 640x640 1 foot, 1 hand, 2 heads, 7.7ms
image 8/25 /content/Climbing-1/test/images/u35_jpg.rf.1101853d11fe31b8279a8ea7750e58d0.jpg: 640x640 2 f

### 2) 모델 구조에 사전 학습 가중치도 가져와서 사용해보기

- 모델 구조 및 사정학습 가중치 선택하기

In [13]:
model2 = YOLO(model='yolov8n.pt', task='detect')

In [14]:
model2.train(data='/content/Climbing-1/data.yaml',
            epochs=150,
            patience=10,
            pretrained=True,
            verbose=True,
            seed=1234,
            )

New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Climbing-1/data.yaml, epochs=150, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=1234, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=Fal

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7933f3a9f850>
fitness: 0.6398193387327723
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.64241,     0.50871,     0.66563])
names: {0: 'foot', 1: 'hand', 2: 'head'}
plot: True
results_dict: {'metrics/precision(B)': 0.973099218404042, 'metrics/recall(B)': 0.9126775883674939, 'metrics/mAP50(B)': 0.9479419432695609, 'metrics/mAP50-95(B)': 0.6055834937842401, 'fitness': 0.6398193387327723}
save_dir: PosixPath('runs/detect/train2')
speed: {'preprocess': 0.23728974011479592, 'inference': 2.5123917326635246, 'loss': 0.0007833753313337054, 'postprocess': 1.9963906735790018}

- 예측해보기

In [15]:
results = model2.predict(source='/content/Climbing-1/test/images/*',
                        save=True,
                        line_width=2,
                        stream=True
                         )

In [16]:
for r in results :
    r_bbox = r.boxes


image 1/25 /content/Climbing-1/test/images/u106_jpg.rf.0e50dad5b2cdf3cb1c705c2a34c924e8.jpg: 640x640 2 foots, 2 hands, 1 head, 13.3ms
image 2/25 /content/Climbing-1/test/images/u110_jpg.rf.78349a8d5d1d2f25d5fcd45eb87d0615.jpg: 640x640 2 hands, 1 head, 9.1ms
image 3/25 /content/Climbing-1/test/images/u112_jpg.rf.a2a70d71b82ed44f4e33a1a0e53e52c5.jpg: 640x640 1 foot, 2 hands, 1 head, 8.3ms
image 4/25 /content/Climbing-1/test/images/u12_jpg.rf.3135fbab1b8ae760b418389e52d94449.jpg: 640x640 2 foots, 2 hands, 2 heads, 8.4ms
image 5/25 /content/Climbing-1/test/images/u21_jpg.rf.4ae71f1edcdb66703710e1496a2e4e1d.jpg: 640x640 4 foots, 1 hand, 1 head, 8.3ms
image 6/25 /content/Climbing-1/test/images/u22_jpg.rf.4eb7505c344be9bca3ea3d56afb76a3d.jpg: 640x640 2 foots, 1 hand, 1 head, 8.1ms
image 7/25 /content/Climbing-1/test/images/u27_jpg.rf.3aa0e2a0c11944115ed8e212f9aefe73.jpg: 640x640 1 foot, 1 hand, 1 head, 10.0ms
image 8/25 /content/Climbing-1/test/images/u35_jpg.rf.1101853d11fe31b8279a8ea7750e5

In [18]:
# # 이미지 한 장 test
# results = model2.predict(source='/content/Climbing-1/test01.jpg',
#                         save=True,
#                         line_width=2,
#                         stream=True
#                          )

# ## 이미지 여러 장 test
# # results = model2.predict(source='/content/Climbing-1/test/*',
# #                         save=True,
# #                         line_width=2,
# #                         stream=True
# #                          )

# ## 영상 test
# # results = model2.predict(source='/content/Climbing-1/test02.mp4',
# #                         save=True,
# #                         line_width=2,
# #                         stream=True
# #                          )

# for r in results :
#     r_bbox = r.boxes